In [1]:
import dask.dataframe as ddf
import pandas as pd
import json
import os
import numpy as np
import hashlib
import re
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from infomap import Infomap
from math import pi

In [2]:
editor_profile = pd.read_parquet('../bibliography/result/bibliography.parquet')

In [4]:
# editor_profile = editor_profile.rename(columns={'Unnamed: 0':'index'}).set_index('index')

In [6]:
all_users = list(editor_profile.columns)

In [7]:
res = {}

In [8]:
np.fill_diagonal(editor_profile.values,0)

In [9]:
def revert_list(user):
    res[user] = editor_profile[user].to_list()

In [10]:
result = [revert_list(i) for i in all_users]

In [11]:
num_users = len(all_users)

In [12]:
sim_matrix = np.empty((num_users,num_users))
# np.fill_diagonal(sim_matrix,np.nan)
np.fill_diagonal(sim_matrix,1)

In [16]:
for i in range(num_users - 1):
    for j in range(i + 1, num_users):
        u1 = all_users[i]
        u2 = all_users[j]
        sim = cosine_similarity([res[u1]],[res[u2]])
        sim_matrix[i][j] = sim
        sim_matrix[j][i] = sim

In [12]:
all_data_sim = pd.DataFrame(sim_matrix,index=all_users,columns=all_users)

In [13]:
# all_data_sim.to_parquet('result/sim-matrix-based-on-editor-profile')

In [14]:
# all_data_sim = pd.read_parquet('result/sim-matrix-based-on-editor-profile')

In [15]:
x = pd.DataFrame(all_data_sim.reset_index().melt('index').loc[lambda x : x['value']> 0].groupby('variable')['value'].apply(list))

In [16]:
x['len'] = x['value'].apply(len)

In [17]:
x = x[x['len']!=1]

In [18]:
all_data_sim = all_data_sim[all_data_sim.index.isin(x.index)]

all_data_sim = all_data_sim[all_data_sim.columns & x.index]

In [19]:
sim_matrix = all_data_sim

In [20]:
sim = sim_matrix.values

In [21]:
sim = sim - 0.01

In [22]:
editor_info = pd.DataFrame(all_users).reset_index().rename(columns={'index':'name',0:'contributor.username'})

In [23]:
res = pd.DataFrame()

In [24]:
def param_test(threshold, method):
    if method == 'sin':
        dis = np.arccos(sim)/pi
    else:
        dis = 1 - sim
    G = nx.from_numpy_matrix(dis) 
    edge_weights = nx.get_edge_attributes(G,'weight')
    G.remove_edges_from((e for e, w in edge_weights.items() if w > threshold))
    im = Infomap(silent=True, num_trials=100,include_self_links=False)   
    mapping = im.add_networkx_graph(G)
    im.run()
    df = im.get_dataframe(columns=["name","node_id", "module_id"])
    num_modules = len(df['module_id'].unique())
    a = df.groupby('module_id').count()
    num_singe_editor_groups = len(a[a['name'] == 1])
    max_ = a['name'].max()
    min_ = a['name'].min()
    greater_than_50 = len(a[a['name']>50])
    ans = 'threshold is {}, number of groups is {}, number of single editor groups is {}, '
    ans += 'max group contains {} editors, {} groups contains more than 50 editors'
    print(ans.format(threshold,num_modules,num_singe_editor_groups,max_,greater_than_50))
    
    fin = df.merge(editor_info,on='name')
    fin = fin.rename(columns={'module_id':'group'}).set_index('contributor.username')[['group']]
    res['threshold-' + str(threshold)] = fin['group']

In [25]:
test = np.arange(0,0.5,0.1).round(2)

In [ ]:
for i in test:
    b = param_test(i,'original')

threshold is 0.0, number of groups is 5295, number of single editor groups is 5295, max group contains 1 editors, 0 groups contains more than 50 editors
threshold is 0.1, number of groups is 1487, number of single editor groups is 1445, max group contains 3120 editors, 6 groups contains more than 50 editors
threshold is 0.2, number of groups is 578, number of single editor groups is 546, max group contains 4479 editors, 2 groups contains more than 50 editors


In [ ]:
# res.to_csv('result/cluster-param-test-by-editor-profile.csv',encoding='utf-16', sep="\t")

In [ ]:
test1 = np.arange(0,0.4,0.02).round(2)